In [20]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [21]:
import os
from pathlib import Path
os.chdir(Path("C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops\\"))

In [22]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [23]:
from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import *
from software_defect_prediction.entity.config_entity import ModelTrainerConfig
from software_defect_prediction.config.configuration import ConfigurationManager

import shutil

In [24]:
from importlib.metadata import files
from sys import exception
import pandas as pd

class Model_Trainer():
    def __init__(self,model_trainer_config : ModelTrainerConfig, predictor_col : str) -> None:
        self.config = model_trainer_config
        self.predictor_col = predictor_col
        self.train_df = None
        
    def prepare_and_load_files(self) -> None:
        try :
            
            files_arr = [self.config.train_file,self.config.test_file]
            
            for file_name in files_arr:
                source_file_path = Path(Path(self.config.source_file_path) / Path(file_name))
                destination_file_path = Path(Path(self.config.root_dir) / Path(file_name))
                if os.path.exists(destination_file_path):
                    os.remove(destination_file_path)

                shutil.copy(source_file_path,self.config.root_dir)

            self.train_df = pd.read_csv(Path(Path(self.config.root_dir) / Path(self.config.train_file)))
            logger.info("input file loaded successfully")
            
        except exception as e:
            logger.error("input file loading failed")
            raise(e)
        

In [27]:
mod_tr = Model_Trainer(ConfigurationManager().get_model_trainer_config(),ConfigurationManager().get_data_schema().TARGET_COLUMN.name)
mod_tr.prepare_and_load_files()
train_df = mod_tr.train_df

2024-05-29 10:21:05.491 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-29 10:21:05.493 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-29 10:21:05.497 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: schema.yaml loaded successfully
2024-05-29 10:21:05.499 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts
2024-05-29 10:21:05.499 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts/model_trainer
2024-05-29 10:21:05.506 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-29 10:21:05.508 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-29 10:21:05.512 | INFO 

In [31]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras_tuner.tuners import RandomSearch
import pandas as pd

# Step 1: Preprocess Data
train_df = pd.read_csv("C:\\Users\kural\\Desktop\\Projects\\End_To_End_MLops\\artifacts\\model_trainer\\train_data.csv") 
test_df = pd.read_csv("C:\\Users\kural\\Desktop\\Projects\\End_To_End_MLops\\artifacts\\model_trainer\\test_data.csv") 

X_train = train_df.drop(columns=['id','defects'])
X_test = test_df.drop(columns=['id','defects'])
y_train = train_df.defects.astype(int)
y_test  = test_df.defects.astype(int)

# Step 2: Define the DNN Model
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))

    # Tune the number of layers and units per layer
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
            activation=hp.Choice('activation', ['relu', 'tanh'])
        ))
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Step 3: Hyperparameter Tuning with Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='hyperparameter_tuning',
    project_name='defect_classification'
)

tuner.search_space_summary()

# Step 4: Perform Hyperparameter Tuning
tuner.search(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Step 5: Evaluate the Best Model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Trial 2 Complete [00h 01m 25s]
val_accuracy: 0.8130132754643759

Best val_accuracy So Far: 0.8130132754643759
Total elapsed time: 00h 02m 28s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
32                |128               |units_0
relu              |relu              |activation
0.00040828        |0.0018127         |learning_rate
256               |192               |units_1

Epoch 1/10


c:\Users\kural\anaconda3\envs\jupyter_nb\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7910 - loss: 0.4973 - val_accuracy: 0.8093 - val_loss: 0.4536
Epoch 2/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8097 - loss: 0.4444 - val_accuracy: 0.8085 - val_loss: 0.4438
Epoch 3/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8102 - loss: 0.4402 - val_accuracy: 0.8095 - val_loss: 0.4512
Epoch 4/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8117 - loss: 0.4394 - val_accuracy: 0.8106 - val_loss: 0.4509
Epoch 5/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8120 - loss: 0.4351 - val_accuracy: 0.8109 - val_loss: 0.4408
Epoch 6/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8125 - loss: 0.4356 - val_accuracy: 0.8126 - val_loss: 0.4389
Epoch 7/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8136 - loss: 0.4335 - val_accuracy: 0.8111 - val_loss: 0.4399
Epoch 8/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8145 - loss: 0.4368 - val_accurac

c:\Users\kural\anaconda3\envs\jupyter_nb\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7860 - loss: 0.5011 - val_accuracy: 0.8020 - val_loss: 0.4497
Epoch 2/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8098 - loss: 0.4418 - val_accuracy: 0.8102 - val_loss: 0.4577
Epoch 3/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8111 - loss: 0.4403 - val_accuracy: 0.8115 - val_loss: 0.4410
Epoch 4/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8129 - loss: 0.4370 - val_accuracy: 0.8123 - val_loss: 0.4572
Epoch 5/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8133 - loss: 0.4391 - val_accuracy: 0.8113 - val_loss: 0.4441
Epoch 6/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8138 - loss: 0.4338 - val_accuracy: 0.8099 - val_loss: 0.4684
Epoch 1/10


c:\Users\kural\anaconda3\envs\jupyter_nb\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7940 - loss: 0.5074 - val_accuracy: 0.8065 - val_loss: 0.4886
Epoch 2/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8096 - loss: 0.4464 - val_accuracy: 0.8112 - val_loss: 0.4397
Epoch 3/10
 652/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8098 - loss: 0.4369